# Educational Tutor Chatbot with Sentiment Analysis

This project develops an **Educational Tutor Chatbot** that adapts its explanations based on the user's emotional state. Utilizing the Llama 2 model for generating responses and a sentiment analysis model from Hugging Face, the chatbot personalizes its teaching style depending on whether the user is feeling frustrated or satisfied. When detecting **negative sentiment** (e.g., frustration), the bot provides simpler explanations, additional examples, or resources to support learning. For **positive sentiment** responses, the chatbot offers more detailed and challenging explanations to foster deeper understanding. The goal is to create a more engaging and responsive learning environment, helping users stay motivated and succeed in their educational journey.

In [ ]:
!pip install -q accelerate protobuf sentencepiece torch git+https://github.com/huggingface/transformers huggingface_hub gradio

In [ ]:
import pandas as pd
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import login
import torch

In [ ]:
# Hugging Face Authentication
login(token="hugging_face_token")

In [ ]:
model_id = "NousResearch/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.use_default_system_prompt = False

In [ ]:
llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
    max_length=1024,
)

sentiment_analyzer = pipeline("sentiment-analysis")

In [ ]:
def answer_question(question):
    """
    Generates an answer to the user's question using Llama 2.

    Args:
    question (str): The input question from the user.

    Returns:
    str: The generated answer.
    """
    response = llama_pipeline(question, max_length=150, do_sample=True)[0]['generated_text']
    response = response.replace(f"Answer: {question}", "").strip()
    return response

def analyze_sentiment(user_input):
    """
    Analyzes the sentiment of the user input.

    Args:
    user_input (str): The input text from the user.

    Returns:
    tuple: A tuple containing the sentiment label and score.
    """
    result = sentiment_analyzer(user_input)[0]
    label = result['label']
    score = result['score']
    return label, score

def educational_tutor_response(question):
    """
    Generates a sentiment-aware response for the user in an educational context.

    Args:
    question (str): The input question from the user.

    Returns:
    str: A sentiment-adjusted response to the question.
    """
    label, score = analyze_sentiment(question)

    # Adjust response based on sentiment
    if label == "NEGATIVE":
        sentiment_adjustment = "It seems you're having some trouble. Let me explain in a simpler way: "
        detailed_help = "Would you like some additional resources on this topic?"
    elif label == "POSITIVE":
        sentiment_adjustment = "Great! I'm glad you're understanding. Here's your answer: "
        detailed_help = "Feel free to ask more if you're curious!"
    else:
        sentiment_adjustment = ""
        detailed_help = ""

    # Generate main response
    main_response = answer_question(question)

    # Provide simpler explanation or encouragement
    full_response = f"{sentiment_adjustment}{main_response}\n{detailed_help}"

    return full_response

In [ ]:
# Gradio Interface
interface = gr.Interface(
    fn=educational_tutor_response,
    inputs="text",
    outputs="text",
    title="Educational Tutor Chatbot with Sentiment Analysis",
    description="Ask a question, and the chatbot will analyze your mood, adjust the response, and provide explanations based on your sentiment.",
    examples=[
        ["I'm really confused about how gravity works."],
        ["I think I understand quantum physics, but I'm not sure."],
        ["Can you explain Pythagoras' theorem?"],
        ["Why is the sky blue?"]
    ],
    theme="huggingface"
)

# Launch the Gradio Interface
interface.launch()